<a href="https://colab.research.google.com/github/Chihiro-n/stockmodel/blob/master/LSTM_Stock_Classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NNで株価を予測モデルを作成する
rev2:入力データを一次元に限定

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import time
from tqdm import tqdm

# LSTMモデルを組む

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pwd

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks

In [ ]:
%ls

In [ ]:
!ps aux
!kill -9 <pid>

# GPUセットのためのおまじない

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# データセットを作る
### PytorchのLSTMに学習させるためのデータは次元の工夫が必要であり、二重リストにしないとエラーになる
### 訓練用とテスト用にデータを分ける。まず二重リストであるtmpを作ってからそれを分割した

In [ ]:
window = 100   # データは日足
input_dim = 4

#csvfile = open('/content/drive/My Drive/data-text.csv', 'r', encoding='utf-8')
#df_train_law = pd.read_csv('./data/stock/label_difflog_2005-2018.csv' ,index_col=0)
df_train_law = pd.read_csv('./data/stock/class_label_difflog_2005-2018.csv' ,index_col=0)

df_train_law.head()

inp_dim = len(df_train_law)

train_endindex = int(inp_dim*0.9)



In [ ]:
print(df_train_law.head())

### inputデータとして、株価をlogに加工した値のみを取り出す。
最初の行だけは削除しておく。階差をlabelとしているため

In [ ]:
if input_dim == 4:

  df_input = df_train_law.iloc[:, 4:8 ]   # 次元が4つの場合
elif input_dim ==1:
  df_input = df_train_law.iloc[:, 4 ]      # 次元を始値に限定する場合

#nd_cast = open.astype(np.float32)

In [ ]:
df_input = df_input.drop('2005-01-05', axis=0)

In [ ]:
df_input

## 念の為、inputdataを確認しておく

In [ ]:
# 
#plt.figure()
#plt.plot(df_input,label="loss")
#plt.legend()
#plt.grid()

In [ ]:
sns.countplot(x="label_class", data=df_train_law)
plt.show()

### inputデータを作成する。
説明変数4つ　が 50日分.  
ループは50を引いておく  
numpyに変更しておく

In [ ]:
#train_input = np.array([])
train_input = []

for i in range(len(df_input)-window):
  #train_input = np.append( train_input, df_input.iloc[i: i+window] )
  train_input.append( df_input.iloc[i: i+window].values )

### 中身チェック

In [ ]:
print(len(train_input))
print(type(train_input))
#train_input[0]
#print(train_input.shape)


### numpyに変更し、サイズもチェック

In [ ]:
np_train_input = np.array(train_input)
np_train_input = np_train_input.astype(np.float32)  ### float型に変更しておかないと、modelに入力する際にerrorが出る　※doubleはNG

print(np_train_input.shape)

np_train_input


## input次元が1のときだけ必要になる。shapeを( len , timestep , 1 )にするため。
※reshapeをしない場合は(len , timestep)だけとなってエラーになる

In [ ]:
np_train_input = np_train_input.reshape(-1,window,input_dim)
print(np_train_input.shape)

# labelデータも numpyに変更しておく
labalはonehot vectorなので、(3366,1,4)になるはず -> X ※pytorchのクロスエントロピーロスはエントロピーロスはonehotvectorに対応しておらずクラスラベルにする

In [ ]:
df_label = df_train_law.iloc[:, 16 ]
df_label
#nd_cast = open.astype(np.float32)

In [ ]:
df_label = df_label.drop('2005-01-05', axis=0)

In [ ]:
type(df_label)

In [ ]:
train_label = []

for i in range(len(df_label)-window):
  #train_label.append( df_label.iloc[window + i].values )
  train_label.append( df_label.iloc[window + i] )

In [ ]:
np_train_label_ = np.array(train_label)
print(np_train_label_)

np_one = np.ones(len(np_train_label_))

np_train_label = np_train_label_ - np_one
print(np_train_label)


np_train_label = np_train_label.astype(np.float32)
np_train_label.shape


## Shapeを変更する、 二重リストの型へ


In [ ]:
#np_train_label = np_train_label.reshape(-1,1)

In [ ]:
print(np_train_label.shape)
print(np_train_label)


## TrainとTestで分割する

In [ ]:
X_train, y_train, X_test, y_test= [], [], [], []


all_length = len(np_train_label)
train_length = int(all_length*0.9)

### 訓練用データセットを作成するループ
X_train = np_train_input[:train_length]
y_train = np_train_label[:train_length]

### テスト用データセットを作成するループ
X_test = np_train_input[train_length:]
y_test = np_train_label[train_length:]


In [ ]:

print(inp_dim)
print( len(X_train[0]) )
print( len(X_train) )

print(X_train[0])
print(X_train[1])
print(X_train[2])

print("=================")
print(y_train[0])
print(y_train[1])

print("=================")
print( len(X_test[0]) )
print( len(X_test) )

print(X_test[0])
print(X_test[1])
print(X_test[2])
print("=================")
print(y_test[0])
print(y_test[1])

In [ ]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)


### dataloader メソッドで、tensorに変更してくれるから必要なし

In [ ]:
"""
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)

X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)
"""

## Dataを組むまでの流れ
1. Datasetクラスを作成する 
    1. 入力データと正解データにわける tensorにすること pd->np->tensorに変換する必要がある
    1. \__len__と\__getitem__を用意すること
1. Dataloaderを準備  
Datasetクラスで作ったやつをinputするだけなので難易度は低い
    

 ### datasetクラスを作成する

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,  X , y , mode='train' ,transform = None):
        self.data = X
        self.label = y
        self.num = len(X)
        self.mode= mode
        self.transform = transform
       
    
    def __len__(self):                         ###データの大きさをreturn するlenメソッドを必ず作成する必要がある
        return self.num
    
    def __getitem__(self, idx):           ###正解データと入力データが対となるようなgetitemメソッドを必ず作成する必要がある
        
        out_data = self.data[idx]
        out_label =  self.label[idx]
        
        return out_data, out_label
        

### XとyをDatasetクラスを継承させて作った自作クラスにinput

In [ ]:
trainset = Dataset( X_train , y_train )

In [ ]:
#trainset.shape
X_train.shape

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset=trainset, batch_size = 32 , shuffle=True)

### 中身の確認
### dataloaderを使用したとしてもshapeは、(バッチ、timestep、説明変数次元)の順になる
dataloaderを使用すると、勝手にnumpy型からtorch型に変更されている。

In [ ]:
dataiter = iter(dataloader)

X, labal = dataiter.next()

In [ ]:
print(X.shape)
print(len(X_train))
print(len(y_train))

# モデルの構築
* input 層
* middle 層
* output 層

In [ ]:
import torch
import torch.nn as nn


class Predictor(nn.Module):
    def __init__(self, inputDim, hiddenDim, outputDim):
        super(Predictor, self).__init__()
        
        self.lstm = nn.LSTM(input_size = inputDim,
                            hidden_size = hiddenDim,
                            batch_first = True)
                           
        self.lstm_output_layer = nn.Linear(hiddenDim, outputDim)
        #self.sigmoid_out = nn.Linear(outputDim, 4)    # labelは one hotベクトルなので outputを4クラスに分けられるようにする
        #self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, inputs, hidden0=None):
        #print(inputs) 　　　　# このinputsには1batchが入る。サイズが4なら 4 ,50, 1 
        output, (hidden, cell) = self.lstm(inputs, hidden0) #LSTM層
        output = self.lstm_output_layer(output[:, -1, :]) #全結合層
        #output = self.sigmoid_out(output)
        return output


In [ ]:
model = Predictor(4,5,4)    # inputの次元は? 株価onlyだから1次元となるのか、それとも時間を5個分とってくるから 5次元？？
#model = Predictor(input_dim,64,4).to(device)     ### 隠れ層は50で行う必要がある
model

In [ ]:
# 損失関数
#criterion = nn.MSELoss()      ### 損失化関数は二乗誤差を設定。　回帰モデルなのでOK
#criterion = nn.CrossEntropyLoss() #ソフトマックス交差エントロピー

criterion = nn.CrossEntropyLoss()



# 最適化関数
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)     ### lrは学習率

In [ ]:
loss_ = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

print(input)
print(input.shape)
print(target)
print(target.shape)
#output_ = loss_(input, target)
#output_.backward()


# dataloaderを使った学習

In [ ]:
ls_loss = []
model.train()


for epoch in range(10):
    
    model.train()
    for i, (X, label) in enumerate(dataloader):

        label = label.long()

        X_torch = X.to(device)
        label_torch = label.to(device)



        #print("X shape ",X_torch.shape)
        #print("label shape",label_torch.shape)
        #print("label",label_torch)
        

        #label_torch = label_torch -1  ## なんの意味があるがよくわかっていないが、損失関数を計算するときにこれを入れないとエラーになる
                                      ## criterion()にて、 index error Target 4 is out of bounds


        # 勾配の初期化
        optimizer.zero_grad()     
        
        y_pred = model( X_torch )          ### ニューラルネットで計算したoutputを保存
        
        #print("y_pred : ",y_pred)
        #print("y_pred shape: ",y_pred.shape)
        #print("label_torch" ,label_torch)
        #print("label_torch shape" ,label_torch.shape)

        loss = criterion(y_pred, label_torch) ###教師データとoutputから損失関数を計算
        #break

        # バッチごとのロスを記録する
        ls_loss.append(loss)
        
        # 勾配の計算
        loss.backward()           ### 損失関数を微分 = 勾配の計算
        
        # パラメータの更新
        optimizer.step()           
        
        #print('*** by optimizer.step()')
    

### ロスの確認 dataloader

In [ ]:
plt.figure()
plt.plot(ls_loss,label="loss")
plt.title('Optimization function error') # 最適化関数の誤差
plt.legend()
plt.grid()

## できあがったmodelでテストを行ってみる

In [ ]:
testset = Dataset( X_test , y_test )
dataloader = torch.utils.data.DataLoader(dataset=testset, batch_size = 1, shuffle=True)

In [ ]:
dataiter = iter(dataloader)

X, labal = dataiter.next()

In [ ]:

y_testresult = []
model.eval()
#t = np.arange(50, 58)



for i, (X, label) in enumerate(dataloader):

    #print("@@@@@@@@@@@@", i)
    
    #print(X)
    optimizer.zero_grad()
    
    y = model(X)
    l = label
    
    #print("label",label)
    #print(y)
    
    y = y.to('cpu').detach().numpy().copy()
    l = l.to('cpu').detach().numpy().copy()
    
    y_flat = y.flatten()
    l_flat = l.flatten()
    
  
    plt.plot(t,y_flat, marker = 'o')
    plt.plot(t,l_flat, marker = 'x')
    plt.show()


In [ ]:
y = model(X)

In [ ]:
print(X)
print(y)
print(l)